# Workshop on Structure-Based Drug Design on a Laptop Spring 2026

# 3.1 Structure Prediction

In this session we will prepare the structure of the protein-ligand complexes using an AI co-folding model called [`Boltz`](https://github.com/jwohlwend/boltz)

Below, you will enter the sequence of the [`HPK1`](https://www.rcsb.org/structure/7L25) protein from the Protein Data Bank and the SMILES strings of the ligands assigned to you to generate the 3D structures of the complexes.

The notebook below is a slightly modified copy of the [`Boltz on Colab`](https://github.com/kimjc95/computational-chemistry) notebook by [`Joo-Chan Kim`](https://github.com/kimjc95).

To run the notebook:

1. Connect to a Google Colab Runtime by clicking on the down-arrow next to the `Connect` button on the upper left, select `Change runtime type`, and selecting a `L4` GPU and `High-RAM`. Then click `Connect`.
2. Execute the first cell to install the required software components.
3. Execute the following cell after entering the name for the job, such as `hpk1-lig1`)
4. At the bottom of the cell, enter the sequence of the protein from the FASTA file of the HPK1 protein and the SMILES of one of the ligands assigned to you in the previous session (`2.1 Retrieve Ligand Smiles from CHEMBL`). Click `Confirm` when done.
5. Scroll down to `Run Prediction` and execute the `Create YAML file from the input data` and `Run prediction using Boltz-2` cells.
6. Finally execute `Download results as a zip file` cell to download the results.
7. Repeat the above for each ligand SMILE string making sure to change the job name accordingly each time.
8. Disconnect the Runtime and scroll down to section `3.2` for the instructions to continue the structure preparation in Maestro.




# Boltz on Colab
2025-06-13 Joo-Chan Kim at [MSBL](https://msbl.kaist.ac.kr), KAIST

This is a Google Colaboratory Notebook for running Boltz-2 at ease (BSD-3 license).

Please cite DOI:[10.5281/zenodo.14881401](https://doi.org/10.5281/zenodo.14881401) if you have used my code in your research.

Please report to [my GitHub](https://github.com/kimjc95/computational-chemistry/issues) or email (kimjoochan@kaist.ac.kr) if you encounter a bug.

Choose a runtime and run below cells one by one.

In [ ]:
#@title Install dependencies
#@markdown Select your runtime type. \
#@markdown GPU runtime with a high capacity RAM is strongly recommended.

runtime = "GPU(L4 or T4)" #@param ["TPU", "GPU(A100)", "GPU(L4 or T4)", "CPU"]

import os
import subprocess

print('Installing dependencies... ', end='')
dependencies = "torch torchvision torchaudio numpy hydra-core pytorch-lightning "
dependencies += "rdkit dm-tree requests pandas types-requests einops einx fairscale "
dependencies += "mashumaro modelcif wandb click pyyaml biopython scipy numba gemmi "
dependencies += "scikit-learn chembl_structure_pipeline "
dependencies += "cuequivariance_ops_cu12 cuequivariance_ops_torch_cu12 cuequivariance_torch"

if runtime == "GPU(L4 or T4)":
    precision = '32-true'
else:
    precision = 'bf16-true'

subprocess.run("pip install ipywidgets torch torchvision torchaudio", shell=True)
subprocess.run("git clone https://github.com/jwohlwend/boltz.git", shell=True)
subprocess.run(f"sed -i 's/bf16-mixed/{precision}/g' /content/boltz/src/boltz/main.py", shell=True)
subprocess.run(f"pip install {dependencies}", shell=True)
subprocess.run("cd boltz; pip install --no-deps -e .", shell=True)

print('done.')

Installing dependencies... done.


# Input data

In [ ]:
#@title Enter sequence input
#@markdown Type the job title name without blanks in the box below.
job_title = "hpk1-lig1" #@param {type:"string"}
#@markdown Run this cell and by using the interactive widgets below, enter the molecule sequence data.

#@markdown For small molecule ligands or modified residues, you can enter the CCD ID (Chemical Compoenent Dictionary code) which can be looked upon the [PDBeChem website](https://www.ebi.ac.uk/pdbe-srv/pdbechem/).

import ipywidgets as widgets
from IPython.display import display, HTML
import os
import re
import requests
from rdkit import Chem, RDLogger
from rdkit.Chem import Draw, AllChem
from google.colab import files
from Bio.PDB import MMCIFParser
from Bio.PDB.Polypeptide import is_aa


def validate_input(text, input_type)->bool:
    """
    Validate the input text about the molecule info based on the specified input type.
    """
    if input_type == 'protein':
        # Use RegEx to check all letters are in 20 canonical amino acid types
        return re.match(r'^[AC-IK-NP-TVWY]+$', text.upper()) is not None
    elif input_type == 'dna':
        # Use RegEx to check all letters are either A, C, G, or T
        return re.match(r'^[ACGT]+$', text.upper()) is not None
    elif input_type == 'rna':
        # Use RegEx to check all letters are either A, C, G, or U
        return re.match(r'^[ACGU]+$', text.upper()) is not None
    elif input_type == 'smiles':
        # Use RDKit to validate the SMILES string
        RDLogger.DisableLog('rdApp.*')
        try:
            mol = Chem.MolFromSmiles(text, sanitize=True)
        except:
            return False
        if mol is None:
            return False
        else:
            return True
    elif input_type == 'ccd':
        # Call header from PDBe website and check HTTP response status
        url = f"https://files.rcsb.org/ligands/download/{text.upper()}_ideal.cif"
        try:
            response = requests.head(url, timeout=5)
            return response.status_code == 200
        except requests.exceptions.RequestException:
            return False


def get_residue_ccd(polymer_type, sequence, index)->str:
    """
    From polymer sequence and index, return the residue's CCD code.
    """
    AA_codes = {
        'A': 'ALA', 'R': 'ARG', 'N': 'ASN', 'D': 'ASP', 'C': 'CYS',
        'E': 'GLU', 'Q': 'GLN', 'G': 'GLY', 'H': 'HIS', 'I': 'ILE',
        'L': 'LEU', 'K': 'LYS', 'M': 'MET', 'F': 'PHE', 'P': 'PRO',
        'S': 'SER', 'T': 'THR', 'W': 'TRP', 'Y': 'TYR', 'V': 'VAL'}

    DNA_codes = {'A': 'DA', 'T': 'DT', 'G': 'DG', 'C': 'DC'}

    if not index:
        return ''

    position = int(index)
    if position < 1 or position > len(sequence):
        return ''

    residue = sequence[position-1].upper()

    if polymer_type == 'protein':
        return AA_codes[residue]
    elif polymer_type == 'dna':
        return DNA_codes[residue]
    else:
        return residue


def get_atom_names(ccd_id)->list:
    """
    Return a list of official atom names from CCD ID.
    """
    try:
        url = f"https://files.rcsb.org/ligands/download/{ccd_id.upper()}_ideal.cif"

        response = requests.get(url)
        response.raise_for_status()

        atom_names = []
        lines = response.text.split('\n')

        in_atom_section = False
        for line in lines:
            if line.startswith('_chem_comp_atom'):
                in_atom_section = True
                continue

            if in_atom_section:
                if line.startswith('#') or line.strip() == '':
                    break
                parts = line.split()
                if len(parts) > 0:
                    if not parts[1].startswith('H'):
                        atom_names.append(parts[1])

        if not atom_names:
             print(f"Could not retrieve atom names for {ccd_id}.")

        return atom_names

    except:
        return []


seq_data = []
mod_data = []
bond_data = []
binder = '' # global variable to save binder chain ID
pocket_data = []
contact_data = []
template_data = []
lig_select = None # global variable to save ligand chain ID used for affinity calculation


def is_it_polymer(chain)->bool:
    """
    Check the given sequence data is a polymer or not.
    """
    for s in seq_data:
        if s['chain'] == chain:
            return s['type'] in ['protein', 'dna', 'rna']
    return False


def return_res_name(chain, index)->str:
    """
    From seq_data and mod_data, return a list of atom names of the given residue.
    """
    for m in mod_data:
        if m['chain'] == chain and m['index'] == index:
            return m['ccd']

    for s in seq_data:
        if s['chain'] == chain:
            return get_residue_ccd(s['type'], s['sequence'], index)

    return ''


def return_atom_name_list(chain, index)->list:
    """
    From seq_data and mod_data, return a list of atom names of the given residue.
    """
    return get_atom_names(return_res_name(chain, index))


def get_polypeptide_chain_ids(cif_file):

    parser = MMCIFParser(QUIET=True)
    try:
        structure = parser.get_structure("protein_structure", cif_file)
    except Exception as e:
        print(f"Error while parsing cif files: {e}")
        return []

    polypeptide_chains = set()

    for model in structure:
        for chain in model:
            for residue in chain:
                if is_aa(residue):
                    polypeptide_chains.add(chain.id)
                    break

    return sorted(list(polypeptide_chains))



class modify_entries():
    """
    Main module to show interactive widgets
    """
    def __init__(self, container):
        self.container = container


    def remove_seq_entry(self, b):
        """
        Operates when '-' button is pressed
        """
        for i in range(1, len(self.container.children)-1):
            # when the minus button is hit
            if self.container.children[i].children[0].children[-1] == b:
                newList = []
                # rename chain IDs in alphabetical order
                for j in range(i+1, len(self.container.children)-1):
                    show_chain = widgets.Label(value= 'chain '+str(chr(ord('A')+j-2)))
                    newline = widgets.HBox([show_chain]+list(self.container.children[j].children[0].children[1:]))
                    newEntry = widgets.VBox([newline]+list(self.container.children[j].children[1:]))
                    newList.append(newEntry)

                self.container.children = list(self.container.children[:i]) + newList + [self.container.children[-1]]
                break


    def add_seq_entry(self, b):
        """
        Operates when '+' button is pressed
        """

        # Chain ID
        show_chain = widgets.Label(value= 'chain '+str(chr(ord('A')+len(self.container.children)-2)))

        # Molecule type
        select_type = widgets.Dropdown(
            options=['protein', 'dna', 'rna', 'smiles', 'ccd'],
            description=' is ',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='120px'))

        # Molecule info string
        enter_sequence = widgets.Text(
            description=' described as :',
            placeholder='MAKEY... or CC1=CC=CC=C1 or ATP',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='600px'))

        # check for cyclic polymers
        cyclic = widgets.Checkbox(
            value=False,
            description=' cyclic? ',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='100px'))

        # minus button
        remove_btn = widgets.Button(description='-', layout=widgets.Layout(width='30px'))
        remove_btn.on_click(self.remove_seq_entry)

        # Error message
        message = widgets.HTML(value='', layout=widgets.Layout(width='600px', padding='5px'))

        def validate_string(change):
            """
            Validate the molecule info entered by the user.
            If the input is incorrect, show the red error message.
            """

            if select_type.value in ['smiles', 'ccd'] and cyclic.value:
                message.value = f"<span style='color: red;'>Only polymers (protein, DNA, and RNA) can be specified as cyclic!</span>"
            elif validate_input(enter_sequence.value, select_type.value):
                message.value = ""
            elif select_type.value in ['protein', 'dna', 'rna']:
                message.value = f"<span style='color: red;'>Enter the valid {select_type.value} sequence.</span>"
            else:
                message.value = f"<span style='color: red;'>Enter the valid {select_type.value} string.</span>"

        enter_sequence.observe(validate_string, names='value')
        select_type.observe(validate_string, names='value')
        cyclic.observe(validate_string, names='value')

        line = widgets.HBox([show_chain, select_type, enter_sequence, cyclic, remove_btn])
        entry = widgets.VBox([line, message])

        self.container.children = list(self.container.children[:-1]) + [entry, self.container.children[-1]]


    def update_seq_data(self, b):
        """
        Operates when the confirm button is pressed
        """
        seq_data.clear()
        for i in range(1, len(self.container.children)-1):
            entry = self.container.children[i]
            line = entry.children[0]
            message = entry.children[1]
            # Filter invalid lines
            if message.value != '':
                continue
            # Filter empty lines
            if line.children[2].value == '':
                continue

            seq = {'chain': line.children[0].value[-1],
                   'type': line.children[1].value,
                   'sequence': line.children[2].value,
                   'cyclic': line.children[3].value,
                   'msa':''}
            seq_data.append(seq)


    def remove_an_entry(self, b):
        """
        Operates when '-' button is pressed, this time on modification / constraint entries
        """
        for i in range(1, len(self.container.children)-1):
            if self.container.children[i].children[0].children[-1] == b:
                self.container.children = list(self.container.children[:i]) + list(self.container.children[i+1:])
                break


    def add_mod_entry(self, b):
        """
        Operates when '+' button is pressed, this time on modification / constraint entries
        """
        polymers = []
        for s in seq_data:
            if s['type'] in ['protein', 'dna', 'rna']:
                polymers.append(s['chain'])

        # select chain
        select_chain = widgets.Dropdown(
            options=polymers,
            description='chain ',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='90px'))

        # enter 1-based residue index
        enter_index = widgets.Text(
            description='position: ',
            placeholder = '0 <',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px'))

        # show the current residue name
        show_res = widgets.Label(value='change from: -', layout=widgets.Layout(width='150px'))

        # read CCD code
        enter_ccd = widgets.Text(
            description='to:',
            placeholder='CCD code',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px'))

        # Minus button
        remove_btn = widgets.Button(description='-', layout=widgets.Layout(width='30px'))
        remove_btn.on_click(self.remove_an_entry)

        # Error message
        message = widgets.HTML(value='', layout=widgets.Layout(width='600px', padding='5px'))

        def validate_string(change):
            """
            Validate the input entered by the user.
            If the input is incorrect, show the red error message.
            """
            error_type = ''
            for s in seq_data:
                if s['chain'] == select_chain.value:
                    res = get_residue_ccd(s['type'], s['sequence'], enter_index.value)
                    if res == '':
                        error_type = 'index'
                        show_res.value = 'change from: -'
                    else:
                        show_res.value = f"change from: {res}"
                    break

            if not validate_input(enter_ccd.value, 'ccd'):
                error_type = 'CCD'

            for m in mod_data:
                if m['chain'] == select_chain.value:
                    if m['index'] == int(enter_index.value):
                        error_type = "duplicate"
                        break

            if error_type == '':
                message.value = ""
            elif error_type == "duplicate":
                message.value = f"<span style='color: red;'>Duplicate residue index.</span>"
            else:
                message.value = f"<span style='color: red;'>Enter the valid {error_type}.</span>"

        select_chain.observe(validate_string, names='value')
        enter_index.observe(validate_string, names='value')
        enter_ccd.observe(validate_string, names='value')

        line = widgets.HBox([select_chain, enter_index, show_res, enter_ccd, remove_btn])
        entry = widgets.VBox([line, message])

        self.container.children = list(self.container.children[:-1]) + [entry, self.container.children[-1]]


    def update_mod_data(self, b):
        """
        Operates when the confirm button is pressed, this time on modification / constraint entries
        """
        mod_data.clear()
        for i in range(1, len(self.container.children)-1):
            entry = self.container.children[i]
            line = entry.children[0]
            message = entry.children[1]
            # Filter out lines with error messages
            if message.value != '':
                continue
            # Filter out empty lines
            if line.children[1].value == '':
                continue
            if line.children[3].value == '':
                continue

            mod = {'chain': line.children[0].value,
                   'index': line.children[1].value,
                   'ccd': line.children[3].value}
            mod_data.append(mod)


    def add_bond_entry(self, b):
        """
        Operates when '+' button is pressed, this time on bond constraints
        """
        # Input for atom1
        select_chain1 = widgets.Dropdown(
            options=[s['chain'] for s in seq_data],
            description='Atom1 chain:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='150px'))

        enter_res1 = widgets.Text(
            description='resi: ',
            placeholder = '> 0',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='150px'))

        show_res1 = widgets.Label(value=" - ", layout=widgets.Layout(width='100px'))

        select_atom1 = widgets.Dropdown(
            description='atom: ',
            options=[],
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px'))

        # Input for atom2
        select_chain2 = widgets.Dropdown(
            options=[s['chain'] for s in seq_data],
            description='Atom2 chain:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='150px'))

        enter_res2 = widgets.Text(
            description='resi: ',
            placeholder = '> 0',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='100px'))

        show_res2 = widgets.Label(value=" - ", layout=widgets.Layout(width='100px'))

        select_atom2 = widgets.Dropdown(
            description='atom: ',
            options=[],
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px'))

        # Minus button
        remove_btn = widgets.Button(description='-', layout=widgets.Layout(width='30px'))
        remove_btn.on_click(self.remove_an_entry)

        # Error message
        message = widgets.HTML(value='', layout=widgets.Layout(width='600px', padding='5px'))

        def validate_string(change):
            """
            Validate the input entered by the user.
            If the input is incorrect, show the red error message.
            """
            error_type = ''

            if select_chain1.value == select_chain2.value and enter_res1.value == enter_res2.value:
                error_type = 'duplicate'
            else:
                show_res1.value = return_res_name(select_chain1.value, enter_res1.value)
                show_res2.value = return_res_name(select_chain2.value, enter_res2.value)

                atom_list1 = return_atom_name_list(select_chain1.value, enter_res1.value)
                atom_list2 = return_atom_name_list(select_chain2.value, enter_res2.value)

                if len(atom_list1) == 0:
                    error_type = 'index1'
                    select_atom1.options = []
                else:
                    select_atom1.options = atom_list1

                if len(atom_list2) == 0:
                    error_type = 'index2'
                    select_atom2.options = []
                else:
                    select_atom2.options = atom_list2

            if error_type == '':
                message.value = ""
            elif error_type == "duplicate":
                message.value = f"<span style='color: red;'>Duplicate residue index.</span>"
            else:
                message.value = f"<span style='color: red;'>Enter the valid number for {error_type}.</span>"

        select_chain1.observe(validate_string, names='value')
        enter_res1.observe(validate_string, names='value')
        select_atom1.observe(validate_string, names='value')
        select_chain2.observe(validate_string, names='value')
        enter_res2.observe(validate_string, names='value')
        select_atom2.observe(validate_string, names='value')

        line = widgets.HBox([select_chain1, enter_res1, show_res1, select_atom1, select_chain2, enter_res2, show_res2, select_atom2, remove_btn])
        entry = widgets.VBox([line, message])

        self.container.children = list(self.container.children[:-1]) + [entry, self.container.children[-1]]


    def update_bond_data(self, b):
        """
        Operates when the confirm button is pressed, this time on bond constraints
        """
        bond_data.clear()
        for i in range(1, len(self.container.children)-1):
            entry = self.container.children[i]
            line = entry.children[0]
            message = entry.children[1]
            # Filter out lines with error
            if message.value != '':
                continue
            # Filter out empty lines
            if line.children[1].value == '':
                continue
            if line.children[5].value == '':
                continue

            bond = {'chain1': line.children[0].value,
                    'index1': line.children[1].value,
                    'atom1': line.children[3].value,
                    'chain2': line.children[4].value,
                    'index2': line.children[5].value,
                    'atom2': line.children[7].value}
            bond_data.append(bond)


    def add_pocket_entry(self, b):
        """
        Operates when '+' button is pressed, this time on pocket constraints
        """
        receptor = []
        for s in seq_data:
            if s['chain'] != binder:
                receptor.append(s['chain'])

        # Reads receptor chain choice
        select_chain = widgets.Dropdown(
            options=receptor,
            description='receptor chain:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='150px'))

        enter_token = None

        # print chosen residue name
        show_res = widgets.Label(value="residue : -", layout=widgets.Layout(width='200px'))

        # Reads max distance in Angstrom
        enter_max_distance = widgets.Text(
            description='max distance (A) : ',
            value='4.0',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px'))

        # Minus button
        remove_btn = widgets.Button(description='-', layout=widgets.Layout(width='30px'))
        remove_btn.on_click(self.remove_an_entry)

        # Error message
        message = widgets.HTML(value='', layout=widgets.Layout(width='600px', padding='5px'))

        def update_token_widget(chain_value):
            """
            Update the token widget based on the type of selected chain.
            """
            nonlocal enter_token # Use nonlocal to modify the outer scope variable

            if is_it_polymer(chain_value):
                enter_token = widgets.Text(
                    description='pocket residue index : ',
                    placeholder = 'resi > 0',
                    style={'description_width': 'initial'},
                    layout=widgets.Layout(width='200px'))

            else:
                # For non-polymers, use atom names from the CCD
                ccd_code = None
                for s in seq_data:
                    if s['chain'] == chain_value and s['type'] == 'ccd':
                        ccd_code = s['sequence']
                        break
                atom_options = []

                if ccd_code:
                    atom_options = get_atom_names(ccd_code)

                enter_token = widgets.Dropdown(
                    description='pocket atom name : ',
                    options=atom_options,
                    style={'description_width': 'initial'},
                    layout=widgets.Layout(width='200px'))

            enter_token.observe(validate_string, names='value') # Observe the new widget


        def validate_string(change):
            """
            Validate the input entered by the user.
            If the input is incorrect, show the red error message.
            """
            error_type = ''
            chain_value = select_chain.value

            if is_it_polymer(chain_value):
                # Validation for polymer (residue index)
                try:
                    index_value = int(enter_token.value)
                    if index_value <= 0:
                        error_type = 'residue index'
                    else:
                        for s in seq_data:
                            if s['chain'] == chain_value:
                                res = get_residue_ccd(s['type'], s['sequence'], enter_token.value)
                                if res == '':
                                    error_type = 'residue index'
                                    show_res.value = 'residue : -'
                                else:
                                    show_res.value = f"residue : {res}"
                                break
                except ValueError:
                    error_type = 'residue index'

            else:
                # Validation for non-polymer (atom name)
                for s in seq_data:
                    if s['chain'] == chain_value:
                        if s['type'] == 'ccd':
                            show_res.value = f"name: {s['sequence']}"
                            break

            for p in pocket_data:
                if p['chain'] == chain_value:
                    if p['token'] == enter_token.value:
                        error_type = "duplicate"
                        break

            try:
                max_d = float(enter_max_distance.value)
                if max_d <= 0:
                    error_type = 'max distance'
            except ValueError:
                error_type = 'max distance'


            if error_type == '':
                message.value = ""
            elif error_type == "duplicate":
                if is_it_polymer(chain_value):
                    message.value = f"<span style='color: red;'>Duplicate residue index found.</span>"
                else:
                    message.value = f"<span style='color: red;'>Duplicate atom name found.</span>"
            elif error_type == 'max distance':
                message.value = f"<span style='color: red;'>Enter the valid {error_type} value in Angstroms.</span>"
            else:
                if is_it_polymer(chain_value):
                    message.value = f"<span style='color: red;'>Enter the valid {error_type}.</span>"
                else:
                    message.value = f"<span style='color: red;'>Enter the valid {error_type} (select from dropdown).</span>"

        # Initialize the enter_token widget based on the initial value of select_chain
        update_token_widget(select_chain.value)

        select_chain.observe(lambda change: update_token_widget(change['new']), names='value')
        enter_max_distance.observe(validate_string, names='value')

        line = widgets.HBox([select_chain, enter_token, show_res, enter_max_distance, remove_btn]) # Use enter_token here
        entry = widgets.VBox([line, message])

        self.container.children = list(self.container.children[:-1]) + [entry, self.container.children[-1]]


    def update_pocket_data(self, b):
        """
        Operates when the confirm button is pressed, this time on pocket constraints
        """
        pocket_data.clear()
        for i in range(1, len(self.container.children)-1):
            entry = self.container.children[i]
            line = entry.children[0]
            message = entry.children[1]
            if message.value != '':
                continue
            if line.children[1].value == '':
                continue # Filter empty text input

            pocket = {'chain': line.children[0].value,
                      'token': line.children[1].value,
                      'max_d': line.children[3].value}
            pocket_data.append(pocket)


    def add_contact_entry(self, b):
        """
        Operates when '+' button is pressed, this time on contact constraints
        """
        # Input for token1
        select_chain1 = widgets.Dropdown(
            options=[s['chain'] for s in seq_data],
            description='chain1 :',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='150px'))

        enter_token1 = None

        # print chosen residue name
        show_res1 = widgets.Label(value="res1: -", layout=widgets.Layout(width='200px'))

        # Input for token2
        select_chain2 = widgets.Dropdown(
            options=[s['chain'] for s in seq_data],
            description='chain2 :',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='150px'))

        enter_token2 = None

        # print chosen residue name
        show_res2 = widgets.Label(value="res2: -", layout=widgets.Layout(width='200px'))

        # Reads max distance in Angstrom
        enter_max_distance = widgets.Text(
            description='max distance (A) : ',
            value='4.0', # defaults to 4.0 A
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px'))

        # Minus button
        remove_btn = widgets.Button(description='-', layout=widgets.Layout(width='30px'))
        remove_btn.on_click(self.remove_an_entry)

        # Error message
        message = widgets.HTML(value='', layout=widgets.Layout(width='600px', padding='5px'))

        def update_token_widgets(chain_values):
            """
            Update the token widget based on the type of selected chain.
            """
            nonlocal enter_token1, enter_token2

            if is_it_polymer(chain_values[0]):
                enter_token1 = widgets.Text(
                    description='resi1: ',
                    placeholder = '> 0',
                    style={'description_width': 'initial'},
                    layout=widgets.Layout(width='200px'))
            else:
                ccd_code = None
                for s in seq_data:
                    if s['chain'] == chain_values[0] and s['type'] == 'ccd':
                        ccd_code = s['sequence']
                        break
                if ccd_code:
                    enter_token1 = widgets.Dropdown(
                        description='atom1: ',
                        options=get_atom_names(ccd_code),
                        style={'description_width': 'initial'},
                        layout=widgets.Layout(width='200px'))

            if is_it_polymer(chain_values[1]):
                enter_token2 = widgets.Text(
                    description='resi2: ',
                    placeholder = '> 0',
                    style={'description_width': 'initial'},
                    layout=widgets.Layout(width='200px'))
            else:
                ccd_code = None
                for s in seq_data:
                    if s['chain'] == chain_values[1] and s['type'] == 'ccd':
                        ccd_code = s['sequence']
                        break
                if ccd_code:
                    enter_token2 = widgets.Dropdown(
                        description='atom2: ',
                        options=get_atom_names(ccd_code),
                        style={'description_width': 'initial'},
                        layout=widgets.Layout(width='200px'))

            enter_token1.observe(validate_string, names='value')
            enter_token2.observe(validate_string, names='value')


        def validate_string(change):
            """
            Validate the input entered by the user.
            If the input is incorrect, show the red error message.
            """
            error_type = ''
            chain_values = [select_chain1.value, select_chain2.value]

            if select_chain1.value == select_chain2.value and enter_token1.value == enter_token2.value:
                error_type = 'duplicate'

            else:
                for i in range(2):
                    chain_value = chain_values[i]
                    token_widget = enter_token1 if i == 0 else enter_token2

                    if is_it_polymer(chain_value):
                        # Validation for polymer (residue index)
                        res = return_res_name(chain_value, token_widget.value)
                        if res == '':
                            error_type = f'residue{i+1} index'
                            show_res_value = f'res{i+1}: -'
                        else:
                            show_res_value = f"res{i+1}: {res}"

                    else:
                        # Validation for non-polymer (atom name)
                        for s in seq_data:
                            if s['chain'] == chain_value:
                                if s['type'] == 'ccd':
                                    show_res_value = f"name: {s['sequence']}"
                                    break
                                else:
                                    show_res_value = f"res{i+1}: -"
                                    break
                    if i == 0:
                        show_res1.value = show_res_value
                    else:
                        show_res2.value = show_res_value

            for c in contact_data:
                if c['chain1'] == chain_values[0] and c['chain2'] == chain_values[1]:
                    if c['token1'] == enter_token1.value and c['token2'] == enter_token2.value:
                        error_type = "duplicate"
                        break

            try:
                max_d = float(enter_max_distance.value)
                if max_d <= 0:
                    error_type = 'max distance'
            except ValueError:
                error_type = 'max distance'

            if error_type == '':
                message.value = ""
            elif error_type == "duplicate":
                message.value = f"<span style='color: red;'>Duplicate info entered.</span>"
            else:
                message.value = f"<span style='color: red;'>Enter the valid number for {error_type}.</span>"

        update_token_widgets([select_chain1.value, select_chain2.value])

        select_chain1.observe(lambda change: update_token_widgets([change['new'], select_chain2.value]), names='value')
        select_chain2.observe(lambda change: update_token_widgets([select_chain1.value, change['new']]), names='value')

        enter_token1.observe(validate_string, names='value')
        enter_token2.observe(validate_string, names='value')
        enter_max_distance.observe(validate_string, names='value')

        line = widgets.HBox([select_chain1, enter_token1, show_res1, select_chain2, enter_token2, show_res2, enter_max_distance, remove_btn])
        entry = widgets.VBox([line, message])

        self.container.children = list(self.container.children[:-1]) + [entry, self.container.children[-1]]


    def update_contact_data(self, b):
        """
        Operates when the confirm button is pressed, this time on pocket constraints
        """
        contact_data.clear()
        for i in range(1, len(self.container.children)-1):
            entry = self.container.children[i]
            line = entry.children[0]
            message = entry.children[1]
            if message.value != '':
                continue
            if line.children[1].value == '':
                continue
            if line.children[4].value == '':
                continue

            contact = {'chain1': line.children[0].value,
                       'token1': line.children[1].value,
                       'chain2' : line.children[3].value,
                       'token2' : line.children[4].value,
                       'max_d': line.children[6].value}
            contact_data.append(contact)


    def add_template_entry(self, b):
        """
        Operates when '+' button is pressed, this time on templates
        """
        # template file
        template_file = widgets.Dropdown(
            options=[t['cif'] for t in template_data],
            description='File name :',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px'))

        protein_chain = widgets.Dropdown(
            options=[s['chain'] for s in seq_data if s['type'] == 'protein'],
            description='Protein chain ID :',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px'))

        template_chain = widgets.Dropdown(
            options=['Auto'],
            description='Template chain ID :',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px'))

        # minus button
        remove_btn = widgets.Button(description='-', layout=widgets.Layout(width='30px'))
        remove_btn.on_click(self.remove_seq_entry)


        def update_chains(change):
            template_chain.options = ['Auto']+get_polypeptide_chain_ids(template_file.value)

        template_file.observe(update_chains, names='value')

        line = widgets.HBox([template_file, protein_chain, template_chain, remove_btn])
        entry = widgets.VBox([line])

        self.container.children = list(self.container.children[:-1]) + [entry, self.container.children[-1]]


    def update_template_data(self, b):
        """
        Operates when the confirm button is pressed
        """
        for i in range(1, len(self.container.children)-1):
            line = self.container.children[i]
            for t in template_data:
                if t['cif'] == line.children[0].value:
                    t['protein'] = line.children[1].value
                    if line.children[2].value != 'Auto':
                        t['template'] = line.children[2].value
                    break


title = widgets.HTML("<h4>Click the plus button to add molecules, and minus button to remove ones. Click the confirm button after entering all entries.</h4>")
add_button = widgets.Button(description='+', layout=widgets.Layout(width='30px'))
confirm_button = widgets.Button(description='confirm', layout=widgets.Layout(width='100px'))
buttons = widgets.HBox([add_button, confirm_button])
seq_container = widgets.VBox([title, buttons])

add_new_seq = modify_entries(seq_container)

add_button.on_click(add_new_seq.add_seq_entry)
confirm_button.on_click(add_new_seq.update_seq_data)
display(seq_container)

## (Optional) Enter modification / constraint data

In [ ]:
#@title Residue-wise Modifications
#@markdown Select the chain, and type the 1-based index for the residue to modify. Type the CCD code for the modified residue.

assert len(seq_data) > 0, "No molecule info entered"
pol_flag = False
for s in seq_data:
    if s['type'] in ['protein', 'dna', 'rna']:
        pol_flag = True
        break
assert pol_flag, "No polymer molecules were entered"

title = widgets.HTML("<h4>Click the plus button to modify biopolymers you entered above, and minus button to remove modifications. Click the confirm button after entering all entries.</h4>")
add_button = widgets.Button(description='+', layout=widgets.Layout(width='30px'))
confirm_button = widgets.Button(description='confirm', layout=widgets.Layout(width='100px'))
buttons = widgets.HBox([add_button, confirm_button])
mod_container = widgets.VBox([title, buttons])

add_new_mod = modify_entries(mod_container)

add_button.on_click(add_new_mod.add_mod_entry)
confirm_button.on_click(add_new_mod.update_mod_data)
display(mod_container)

AssertionError: No molecule info entered

In [ ]:
#@title Bond Constraints
#@markdown Only bonds between two atoms from biopolymers can be entered. \
#@markdown Select the the atom's chain ID, enter 1-based residue index, and then select the atom from the dropdown menu.

assert len(seq_data) > 0, "No molecule info entered"

title = widgets.HTML("<h4>Click the plus button to add covalent bond constraints to the molecules, and minus button to remove constraints. Click the confirm button after entering all entries.</h4>")
add_button = widgets.Button(description='+', layout=widgets.Layout(width='30px'))
confirm_button = widgets.Button(description='confirm', layout=widgets.Layout(width='100px'))
buttons = widgets.HBox([add_button, confirm_button])
bond_container = widgets.VBox([title, buttons])

add_new_bond = modify_entries(bond_container)

add_button.on_click(add_new_bond.add_bond_entry)
confirm_button.on_click(add_new_bond.update_bond_data)
display(bond_container)

In [ ]:
#@title Pocket Constraints
#@markdown First select the chain ID of the molecule of interest in the pocket (binder). \
#@markdown Then enter the info for the residue/atoms that constitute the pocket. \
#@markdown You can also set the maximum distance between the binder and the pocket in Angstroms. \
#@markdown Note that only one binder per system is supported.

assert len(seq_data) > 1, "Less than 2 molecules entered"

title = widgets.HTML("<h4>Choose the binder chain. Then add the pocket constraints. Click the confirm button after adding all entries.</h4>")
choose_ligand = widgets.Dropdown(
    options=[s['chain'] for s in seq_data],
    description='binder chain ID:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='150px'))
add_button = widgets.Button(description='+', layout=widgets.Layout(width='30px'))
confirm_button = widgets.Button(description='confirm', layout=widgets.Layout(width='100px'))
header = widgets.VBox([title, choose_ligand])
buttons = widgets.HBox([add_button, confirm_button])
pocket_container = widgets.VBox([header, buttons])

def update_binder(change):
    global binder
    binder = change['new']

choose_ligand.observe(update_binder, names='value')

add_new_pocket = modify_entries(pocket_container)
add_button.on_click(add_new_pocket.add_pocket_entry)
confirm_button.on_click(add_new_pocket.update_pocket_data)
display(pocket_container)

In [ ]:
#@title Contact Constraints
#@markdown Select a pair of residue / atom and set the maximum distance between them.

assert len(seq_data) > 0, "No molecule info entered"

title = widgets.HTML("<h4>Click the plus button to add contact constraints to the molecules, and minus button to remove constraints. Click the confirm button after entering all entries.</h4>")
add_button = widgets.Button(description='+', layout=widgets.Layout(width='30px'))
confirm_button = widgets.Button(description='confirm', layout=widgets.Layout(width='100px'))
buttons = widgets.HBox([add_button, confirm_button])
contact_container = widgets.VBox([title, buttons])

add_new_contact = modify_entries(contact_container)
add_button.on_click(add_new_contact.add_contact_entry)
confirm_button.on_click(add_new_contact.update_contact_data)
display(contact_container)

## (Optional) Upload template / custom MSA files for proteins

In [ ]:
#@markdown Run this cell to upload template cif files. You can select multiple files.

assert seq_data, "No molecule info entered"

proteins = [s for s in seq_data if s['type'] == 'protein']
assert proteins, "No protein molecules entered! Templates are currently only applicable to proteins."


cif_files = files.upload()
template_data.clear()

for c in list(cif_files.keys()):
    chainIDs = get_polypeptide_chain_ids(c)
    if len(chainIDs) > 0:
        template_data.append({'cif':c, 'protein':None, 'template':None})

assert template_data, "No polypeptide chains were found in the uploaded cif files."

title = widgets.HTML("<h4>Uploaded template files are automatically matched in default. To manually set the chain ID click the plus button below.</h4>")
add_button = widgets.Button(description='+', layout=widgets.Layout(width='30px'))
confirm_button = widgets.Button(description='confirm', layout=widgets.Layout(width='100px'))
buttons = widgets.HBox([add_button, confirm_button])
template_container = widgets.VBox([title, buttons])

add_new_template = modify_entries(template_container)

add_button.on_click(add_new_template.add_template_entry)
confirm_button.on_click(add_new_template.update_template_data)
display(template_container)

In [ ]:
#@markdown Upload custom MSA a3m files and run this cell to allocate them.

assert len(seq_data) > 0, "No molecule info entered"

proteins = [s for s in seq_data if s['type'] == 'protein']
assert len(proteins) > 0, "No protein molecules entered! MSAs are currently only applicable to proteins."

title = widgets.HTML("<h4>Upload your MSA files and enter the file name next to the corresponding polypeptide chain.</h4>")

p_chains = []
for p in proteins:
    cyclic = 'cyclic' if p['cyclic'] else 'linear'
    chain = widgets.Label(value=f"chain ID : {p['chain']}  length : {len(p['sequence'])} aa, {cyclic}", layout=widgets.Layout(width='500px'))
    filename = widgets.Text(description='MSA file name : ', value='.a3m', style={'description_width': 'initial'}, layout=widgets.Layout(width='300px'))
    p_chains.append(widgets.HBox([chain, filename]))

table = widgets.VBox(p_chains)

confirm_button = widgets.Button(description='confirm', layout=widgets.Layout(width='100px'))

def update_msa_data(b):
    for i, p in enumerate(proteins):
        fname = table.children[i].children[1].value.split('.')[0]
        if fname == '':
            continue
        else:
            p['msa'] = f"/content/{fname}.a3m"

confirm_button.on_click(update_msa_data)
display(title, table, confirm_button)

## (Optional) Predict binding affinity

In [ ]:
assert seq_data, "No molecule info entered"

ligands = [s for s in seq_data if s['type'] in ['smiles', 'ccd']]
assert ligands, "No small molecules entered! Binding affinity predictions cannot be applied to proteins and nucleic acids."

import io

#@title Set the target ligand for affinity prediction

#@markdown Number of diffusion samples to be used for affinity prediction. (default: 5)
diffusion_samples_affinity = 5 #@param {type:"slider", min:1, max:10, step:1}
#@markdown Number of sampling steps for affinity prediction. (default: 200)
sampling_steps_affinity = 200 #@param {type:"slider", min:50, max:400, step:50}

lig_select = widgets.Dropdown(options=['None']+[l['chain'] for l in ligands],
                              value='None',
                              style={'description_width': 'initial'},
                              description='Ligand chain ID:',
                              layout=widgets.Layout(width='300px'))

lig_image = widgets.Image(value=b'', format='png', width=400, height=300)

def update_sdf(change): # callback function to interactively update viewer
    ligand = lig_select.value
    if ligand == 'None':
        return
    for s in seq_data:
        if s['chain'] == ligand:
            RDLogger.DisableLog('rdApp.*')
            if s['type'] == 'smiles':
                mol = Chem.MolFromSmiles(s['sequence'], sanitize=True)
            elif s['type'] == 'ccd':
                ccd = s['sequence'].upper()
                if not os.path.exists(ccd+'_ideal.sdf'):
                    subprocess.run(f'wget https://files.rcsb.org/ligands/download/{ccd}_ideal.sdf', shell=True)
                sdf_file = f'{ccd}_ideal.sdf'
                mol = Chem.rdmolfiles.SDMolSupplier(sdf_file)[0]
            break

    AllChem.Compute2DCoords(mol)
    img = Draw.MolToImage(mol, size=(400, 300))

    with io.BytesIO() as output:
        img.save(output, format="PNG")
        lig_image.value = output.getvalue()

lig_select.observe(update_sdf, names='value')

print("Select the ligand's chain ID and check the structure. Select None to cancel.")

display(lig_select, lig_image)

# Run Prediction

In [ ]:
#@title Create YAML file from the input data
#@markdown Once you are confident with all the inputs above, run this cell to generate the input file.

#@markdown The created .yaml file can be downloaded, edited and uploaded for faster setup in the future.

import yaml

data = {'version': 1, 'sequences':[]}

for s in seq_data:
    if s['type'] in ['protein', 'dna', 'rna']:
        seq = {s['type']:{'id':s['chain'], 'sequence':s['sequence']}}
        mod = []
        for m in mod_data:
            if m['chain'] == s['chain']:
                mod.append({'position':int(m['index']), 'ccd':m['ccd']})
        if mod:
            seq[s['type']]['modifications'] = mod
        if s['type'] == 'protein' and s['msa'] != '':
            seq['protein']['msa'] = s['msa']
        seq[s['type']]['cyclic'] = s['cyclic']
    elif s['type'] == 'smiles':
        seq = {'ligand':{'id':s['chain'], 'smiles':s['sequence']}}
    elif s['type'] == 'ccd':
        seq = {'ligand':{'id':s['chain'], 'ccd':s['sequence']}}
    data['sequences'].append(seq)

if (len(bond_data)+len(pocket_data)+len(contact_data)) >0:
    data['constraints'] = []

    for bond in bond_data:
        data['constraints'].append({'bond':{'atom1':[bond['chain1'], int(bond['index1']), bond['atom1']], 'atom2':[bond['chain2'], int(bond['index2']), bond['atom2']]}})

    if len(pocket_data) > 0:
        data['constraints'].append({'pocket':{'binder':binder, 'contacts':[], 'max_distance':''}})

    max_distance = 0.0
    for pocket in pocket_data:
        if is_it_polymer(pocket['chain']):
            token = int(pocket['token'])
        else:
            token = pocket['token']
        data['constraints'][-1]['pocket']['contacts'].append([pocket['chain'], token])
        max_d = float(pocket['max_d'])

        if max_d > max_distance:
            max_distance = max_d

    data['constraints'][-1]['pocket']['max_distance'] = max_distance

    for contact in contact_data:
        if is_it_polymer(contact['chain1']):
            token1 = int(contact['token1'])
        else:
            token1 = contact['token1']
        if is_it_polymer(contact['chain2']):
            token2 = int(contact['token2'])
        else:
            token2 = contact['token2']

        data['constraints'].append({'contact':{'token1':[contact['chain1'], token1], 'token2':[contact['chain2'], token2], 'max_distance':float(contact['max_d'])}})

if len(template_data) >0:
    data['templates'] = []
    for t in template_data:
        temp = {'cif':'/content/'+t['cif']}
        if t['protein'] is not None:
            temp['chain_id'] = t['protein']
        if t['template'] is not None and t['template'] != 'Auto':
            temp['template_id'] = t['template']
        data['templates'].append(temp)

if lig_select is not None and lig_select.value != 'None':
    data['properties'] = [{'affinity':{'binder':lig_select.value}}]

with open(f'{job_title}.yaml', 'w') as f:
    yaml.dump(data, f, default_flow_style=False, sort_keys=False)
    print('Done!')

Done!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.environ['TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD'] = '1'
#@title Run prediction using Boltz-2
#@markdown Output format
output_format = 'mmcif' #@param ["pdb", "mmcif"]
#@markdown Number of dataloader workers
num_workers = 2 #@param {type:"slider", min:0, max:5, step:1}
#@markdown Lower the step scale to increase the diversity of result. (default: 1.638)
step_scale = 1.638 #@param {type:"slider", min:1, max:2, step:0.001}
#@markdown Number of diffusion samples to be generated. (default: 1, AlphaFold3: 5)
diffusion_samples = 4 #@param {type:"slider", min:1, max:10, step:1}
#@markdown Number of recycling steps for the prediction. (default: 3, AlphaFold3: 10)
recycling_steps = 3 #@param {type:"slider", min:1, max:25, step:1}
#@markdown Number of sampling steps for structure prediction. (default: 200)
sampling_steps = 50 #@param {type:"slider", min:50, max:400, step:50}
#@markdown Maximum number of MSA sequences to be used
max_msa_seqs = 8192 #@param [32, 64, 128, 256, 512, 1024, 2048, 4096, 8192]
#@markdown Subsample MSA?
subsample_msa = False #@param {type:"boolean"}
#@markdown Number of subsampled MSA
num_subsampled_msa = 1024 #@param [4, 8, 16, 32, 64, 128, 256, 512, 1024]
#@markdown MSA pairing strategy
msa_pairing_strategy = 'greedy' #@param ['greedy', 'complete']

use_msa_server = False
for s in seq_data:
    if s['type'] == 'protein' and s['msa'] == '':
        use_msa_server = True
        break



commandline = f'{job_title}.yaml --num_workers {num_workers} --step_scale {step_scale} --recycling_steps {recycling_steps} --sampling_steps {sampling_steps}'
commandline += f' --diffusion_samples {diffusion_samples} --max_msa_seqs {max_msa_seqs} --msa_pairing_strategy {msa_pairing_strategy}'
if subsample_msa:
    commandline += f' --num_subsampled_msa {num_subsampled_msa}'
if use_msa_server:
    commandline += ' --use_msa_server'
if lig_select is not None and lig_select.value != 'None':
    commandline += f' --diffusion_samples_affinity {diffusion_samples_affinity} --sampling_steps_affinity {sampling_steps_affinity}'
if runtime.startswith('GPU'):
    commandline += ' --no_kernels --accelerator gpu'
elif runtime == 'TPU':
    commandline += ' --accelerator tpu'
else:
    commandline += ' --accelerator cpu'

!boltz predict {commandline} --out_dir /content/{job_title}

In [ ]:
#@title Download results as a zip file

import zipfile
from google.colab import files

print("Downloading result files.")
filename = f'{job_title}.zip'

with zipfile.ZipFile(filename, 'w') as zip_file:
    dir_path = f'/content/{job_title}/boltz_results_{job_title}'
    for root, directory, items in os.walk(dir_path):
        for item in items:
            path = os.path.join(root, item)
            zip_file.write(path, arcname=os.path.relpath(os.path.join(root, item), dir_path), compress_type=zipfile.ZIP_DEFLATED)

files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title (Optional) Calculate Ligand Kd and dG

assert lig_select is not None, "No ligand selected"

import json
import math

#@markdown Set the temperature in Celsius.
temperature = 25 #@param {type:"slider", min:0, max:100, step:1}

file_path = f'/content/{job_title}/boltz_results_{job_title}/predictions/{job_title}/affinity_{job_title}.json'

with open(file_path, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

    pred_value = raw_data['affinity_pred_value']
    Kd = math.pow(10, pred_value)/1000000
    dG = 8.3144626*(273.15+temperature)*math.log(Kd)/1000

print(f'Predicted dissociation eq constant  : {Kd:.3e} mol/L  (= {Kd*1000000:.1f} uM)')
print(f'Predicted binding Gibbs free energy : {dG:.3f} kJ/mol')
print(f'                                      {dG/4.184:.3f} kcal/mol')

# Tools used

* Boltz-2
> S. Passaro, G. Corso, J. Wohlwend, M. Reveiz, S. Thaler, V. R. Somnath, N. Getz, T. Portnoi, J. Roy, H. Stark, D. Kwabi-Addo, D. Beaini, T. Jaakkola, and R. Barzilay (2025) "Boltz-2: Towards Accurate and Efficient Binding Affinity Prediction." bioRxiv.

* Boltz-1
> J. Wohlwend, G. Corso, S. Passaro, M. Reveiz, K. Leidal, W. Swiderski,  T. Portnoi, I. Chinn, J. Silterra, T. Jaakkola, and R. Barzilay (2024) "Boltz-1: Democratizing Biomolecular Interaction Modeling." bioRxiv. DOI:[10.1101/2024.11.19.624167](https://doi.org/10.1101/2024.11.19.624167)

* ColabFold
> M. Mirdita, K. Schütze, Y. Moriwaki, L. Heo, S. Ovchinnikov, and M. Steinegger (2022) "ColabFold: making protein folding accessible to all" Nature methods, 19, 679-682. DOI:[10.1038/s41592-022-01488-1](https://doi.org/10.1038/s41592-022-01488-1)

*  RDKit
> RDKit: Open-source cheminformatics. https://www.rdkit.org


# 3.2 Structure Preparation in Maestro

1. On the laptop, start Maestro in the working directory we used earlier in Step `2.1` and open the project we set up (`hpk1.prj`)
2. unzip one of the Boltz prediction zip files in a temporary folder.
3. Import the Bolt-predicted structure. Find it under `predictions/JOBNAME/JOBNAME_model_0.cif`, where `JOBNAME` is the job name you picked when predicted the structure above; `hpk1-lig1` for example.
4. Compare the Boltz-predicted structure with one of the deposited PDB crystal structures of this system, `7L25`, for example.
      *    Download the `7L25` PDB strucure using the `File--Get PDB` feature.
      *    Split the structure into chains.
      *    Superimpose the protein structure of the Boltz-predicted structure with one of the chains of the PDB structure using the `Superposition--Calpha-atoms` Task. How different are the two structures?
      *    Split the structures into protein, ligand, etc.
      *    Are the two ligands well aligned? Measure the RMSD deviation between the two ligands using the `Superposition--Maximum Common Substructure` Task.
      *    Are the main protein-ligand interactions present in the predicted structure?
6. Double-check the bond orders and formal charges of the ligand. Fix any issues.
7. Add hydrogen atoms to the receptor structure and the ligand using the `Build--Add Hydrogens` feature.
8. Export the protein structure to a `.pdb` file and the ligand to a `.sdf` file.
9. Join the protein and ligand structure into a new workspace entry using `Workspace--Create Entry` feature. Export the structure of the complex into a `.pdb` file.
10. Repeat the above with the Boltz prediction for the second ligand.

